In [1]:
import torch
from torch import nn
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
import sys
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.optim as optim
import gc
import os
from torchvision.ops import batched_nms
import importlib

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import utils
import loss
importlib.reload(loss)
importlib.reload(utils)

<module 'utils' from 'd:\\YoloV3\\utils.py'>

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
DATASET = 'D:/YoloV3/Seeds' # change to fit your path
DEVICE = device
# seed_everything()  # If you want deterministic behavior
NUM_WORKERS = 2
BATCH_SIZE = 8
IMAGE_SIZE = 416
NUM_CLASSES = 2
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0
NUM_EPOCHS = 25
CONF_THRESHOLD = 0.5
MAP_IOU_THRESH = 0.5
NMS_IOU_THRESH = 0.45
S = [IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8]
PIN_MEMORY = True
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_FILE = "plz.pth.tar"
IMG_DIR = DATASET + "/images/"
LABEL_DIR = DATASET + "/labels/"
train_csv_path = DATASET + "/train.csv"
test_csv_path = DATASET + "/val.csv"


ANCHORS = [
    [(0.28, 0.21), (0.2, 0.15), (0.14, 0.12)],
    [(0.0650, 0.0671), (0.0374, 0.0485), (0.0277, 0.0558)],
    [(0.0309, 0.0272), (0.0292, 0.0293), (0.0121, 0.0151)],
]  # Note these have been rescaled to be between [0, 1]


scale = 1.1
train_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=int(IMAGE_SIZE * scale)),
        A.PadIfNeeded(
            min_height=int(IMAGE_SIZE * scale),
            min_width=int(IMAGE_SIZE * scale),
            border_mode=cv2.BORDER_CONSTANT,
            value = 0
        ),
        A.RandomCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        A.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
        A.OneOf(
            [
                A.ShiftScaleRotate(
                    rotate_limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT
                ),
                A.Affine(shear=15, p=0.5),
            ],
            p=1.0,
        ),
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.1),
        A.CLAHE(p=0.1),
        A.Posterize(p=0.1),
        A.ToGray(p=0.1),
        A.ChannelShuffle(p=0.05),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
)

test_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=IMAGE_SIZE),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value = 0
        ),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[]),
)

SEEDS_CLASSES = [
    ".",
    ","
]

PASCAL_CLASSES = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor"
]

In [5]:
from utils import cells_to_bboxes, non_max_suppression
from PIL import Image

def plot_image(image, boxes):
    """Plots predicted bounding boxes on the image"""
    cmap = plt.get_cmap("tab20b")
    class_labels = SEEDS_CLASSES
    colors = [cmap(i) for i in np.linspace(0, 1, len(class_labels))]
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # box[0] is x midpoint, box[2] is width
    # box[1] is y midpoint, box[3] is height

    # Create a Rectangle patch
    for box in boxes:
        assert len(box) == 6, "box should contain class pred, confidence, x, y, width, height"
        class_pred = box[0]
        box = box[2:]
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=2,
            edgecolor=colors[int(class_pred)],
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)
        plt.text(
            upper_left_x * width,
            upper_left_y * height,
            s=class_labels[int(class_pred)],
            color="white",
            verticalalignment="top",
            bbox={"color": colors[int(class_pred)], "pad": 0},
        )
    plt.show()
    
def plot_couple_examples(model, loader, thresh, iou_thresh, anchors):
    model.eval()
    x, y = next(iter(loader))
    x = x.to("cuda")
    with torch.inference_mode():
        out = model(x)
        bboxes = [[] for _ in range(x.shape[0])]
        for i in range(3):
            batch_size, A, S, _, _ = out[i].shape
            anchor = anchors[i]
            boxes_scale_i = cells_to_bboxes(
                out[i], anchor, S=S, is_preds=True
            )
            for idx, (box) in enumerate(boxes_scale_i):
                bboxes[idx] += box

        model.train()

    for i in range(batch_size):
        nms_boxes = non_max_suppression(
            bboxes[i], iou_threshold=iou_thresh, threshold=thresh, box_format="midpoint",
        )
        plot_image(x[i].permute(1,2,0).detach().cpu(), nms_boxes)

def plot_one_image(model, image_path, thresh, iou_thresh, anchors):
    model.eval()
    image = np.array((Image.open(image_path)).convert("RGB"))
    augmentations = test_transforms(image = image)
    image = augmentations["image"].unsqueeze(0).to(DEVICE)
    with torch.inference_mode():
        out = model(image)
        bboxes = [[] for _ in range(image.shape[0])]
        for i in range(3):
            _, _, S, _, _ = out[i].shape
            anchor = anchors[i]
            boxes_scale_i = cells_to_bboxes(
                out[i], anchor, S=S, is_preds=True
            )
            for idx, (box) in enumerate(boxes_scale_i):
                bboxes[idx] += box

        model.train()

    nms_boxes = non_max_suppression(
        bboxes[0], iou_threshold=iou_thresh, threshold=thresh, box_format="midpoint",
    )
    plot_image(image[0].permute(1,2,0).detach().cpu(), nms_boxes)

In [6]:
from model import YOLOv3

In [7]:

def test_model():
    model = YOLOv3(num_classes=NUM_CLASSES)
    x = torch.randn((2, 3, IMAGE_SIZE, IMAGE_SIZE))
    out = model(x)
    assert model(x)[0].shape == (2, 3, IMAGE_SIZE//32, IMAGE_SIZE//32, NUM_CLASSES + 5)
    print(out[0].shape)
    assert model(x)[1].shape == (2, 3, IMAGE_SIZE//16, IMAGE_SIZE//16, NUM_CLASSES + 5)
    assert model(x)[2].shape == (2, 3, IMAGE_SIZE//8, IMAGE_SIZE//8, NUM_CLASSES + 5)
    print("Success!")

In [8]:
# test_model()

In [9]:
from dataset import YOLODataset

train_dataset = YOLODataset(
    train_csv_path,
    transform=train_transforms,
    S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    anchors=ANCHORS,
)
test_dataset = YOLODataset(
    test_csv_path,
    transform=test_transforms,
    S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    anchors=ANCHORS,
)

In [10]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=False,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=False,
    drop_last=False,
)

In [11]:
# test_X, test_y = next(iter(train_loader))
# print(test_X.shape)
# print(test_y[0].shape)

In [12]:
torch.backends.cudnn.benchmark = True

In [13]:
from loss import YoloLoss
from utils import check_class_accuracy, get_evaluation_bboxes, mean_average_precision

In [ ]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr, device):
    print(f"=> Loading checkpoint {checkpoint_file}")
    checkpoint = torch.load(checkpoint_file, map_location= device, weights_only=True)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [ ]:
def train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors, schedular=None, device=DEVICE):
    # losses = []
    model.train()
    pbar = tqdm(enumerate(train_loader), leave = True)
    for batch_idx, (X, y) in pbar:
        X = X.to(device)
        y0, y1, y2 = (
            y[0].to(device),
            y[1].to(device),
            y[2].to(device)
        )

        with torch.amp.autocast(device_type = DEVICE):
            output = model(X)
            loss = (
                loss_fn(output[0], y0, scaled_anchors[0]) 
                + loss_fn(output[1], y1, scaled_anchors[1]) 
                + loss_fn(output[2], y2, scaled_anchors[2]) 
            )
        
        # losses.append(loss.item())
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        schedular.step()
        torch.cuda.empty_cache()
        gc.collect()

        # mean_loss = sum(losses) / len(losses)
        pbar.set_postfix(loss = loss.item())

In [16]:
model = YOLOv3(num_classes=NUM_CLASSES).to(DEVICE)
optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
schedular = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95) # lr = lr * gamma ^ epoch
loss_fn = YoloLoss()
scaler = torch.amp.GradScaler()

if LOAD_MODEL:
    load_checkpoint(
        CHECKPOINT_FILE, model, optimizer, LEARNING_RATE, DEVICE
    )

scaled_anchors = (
    torch.tensor(ANCHORS)
    * torch.tensor(S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
).to(DEVICE)

=> Loading checkpoint plz.pth.tar


In [17]:
# plot_one_image(model, test_path, 0.65, 0.25, scaled_anchors)

In [19]:
check_class_accuracy(model, test_loader, threshold=CONF_THRESHOLD, device = DEVICE)

100%|██████████| 12/12 [00:19<00:00,  1.60s/it]

Class accuracy is: 89.114471%
No obj accuracy is: 95.815063%
Obj accuracy is: 76.175728%


tensor(82.6451, device='cuda:0')

In [ ]:
labels_path = "Seeds/labels"

invalid_txt = []
for txtfile in os.listdir(labels_path):
    if txtfile.endswith(".txt"):
        with open(os.path.join(labels_path, txtfile), 'r') as f:
            valid_file = True
            for line in f:
                try:
                    _, x, y, w, h = map(float, line.split())
                    # print(x,y,w,h)
                    if w == 0 or h == 0:
                        print(x,y,w,h)
                        valid_file = False
                        break
                except ValueError:
                    valid_file = False
                    break
            if not valid_file:
                invalid_txt.append(txtfile)
                
invalid_txt

[]

In [20]:
best_acc = 0
for epoch in range(NUM_EPOCHS):
    train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors, schedular, DEVICE)
    acc = check_class_accuracy(model, test_loader, threshold=CONF_THRESHOLD, device = DEVICE)
    if acc > best_acc:
        print(f"saving new model")
        save_checkpoint(model, optimizer, filename=f"plz2.pth.tar")
        best_acc = acc
    # if epoch > 0 and (epoch + 1) % 5 == 0:
    #     check_class_accuracy(model, test_loader, threshold=CONF_THRESHOLD, device = DEVICE)
    #     print("start inference")
    #     pred_boxes, true_boxes = get_evaluation_bboxes(
    #         test_loader,
    #         model,
    #         iou_threshold=NMS_IOU_THRESH,
    #         anchors=ANCHORS,
    #         threshold=CONF_THRESHOLD,
    #     )
    #     print("start map")
    #     mapval = mean_average_precision(
    #         pred_boxes,
    #         true_boxes,
    #         iou_threshold=MAP_IOU_THRESH,
    #         box_format="midpoint",
    #         num_classes=NUM_CLASSES,
    #     )
    #     print(f"epoch {epoch} MAP: {mapval.item()}")
    #     model.train()
    #     if SAVE_MODEL and mapval.item() > best_map:
    #         print("saving new best model")
    #         save_checkpoint(model, optimizer, filename=f"checkpoint.pth.tar")
    #         best_map = mapval.item()
    


0it [00:00, ?it/s]

train loss
box 0.345595121383667
obj 0.19893379509449005
noobj 0.7371559143066406
class 1.4044878482818604

train loss
box 0.13741248846054077
obj 0.13488440215587616
noobj 1.8083863258361816
class 1.2624198198318481

train loss
box 0.15337443351745605
obj 0.09703249484300613
noobj 1.6821956634521484
class 1.2357678413391113



c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


train loss
box 0.37758398056030273
obj 0.20474731922149658
noobj 0.7571297287940979
class 1.3991787433624268

train loss
box 0.1596398800611496
obj 0.1251765936613083
noobj 1.7967774868011475
class 1.4264848232269287

train loss
box 0.15638665854930878
obj 0.10902662575244904
noobj 1.6762146949768066
class 1.379256248474121

train loss
box 0.4289063811302185
obj 0.2587926685810089
noobj 0.7606596350669861
class 1.3658316135406494

train loss
box 0.2136075794696808
obj 0.16440846025943756
noobj 1.8231003284454346
class 1.259749412536621

train loss
box 0.17943638563156128
obj 0.12508819997310638
noobj 1.692332148551941
class 1.2113419771194458

train loss
box 0.3848811388015747
obj 0.25789740681648254
noobj 0.7201910614967346
class 1.050947904586792

train loss
box 0.18317225575447083
obj 0.14494909346103668
noobj 1.765836238861084
class 0.8946324586868286

train loss
box 0.15948760509490967
obj 0.10047923028469086
noobj 1.6665573120117188
class 0.7923651337623596

train loss
box 0.4228

100%|██████████| 12/12 [00:20<00:00,  1.73s/it]


...

Class accuracy is: 89.045647%
No obj accuracy is: 98.617424%
Obj accuracy is: 59.428768%


0it [00:00, ?it/s]

train loss
box 0.3417285680770874
obj 0.3012428879737854
noobj 0.44125932455062866
class 1.178794264793396

train loss
box 0.17145869135856628
obj 0.26603442430496216
noobj 1.0738316774368286
class 1.0084178447723389

train loss
box 0.13391312956809998
obj 0.19105517864227295
noobj 1.2843841314315796
class 0.9063061475753784

train loss
box 0.32915717363357544
obj 0.27634507417678833
noobj 0.4423540532588959
class 1.1005998849868774

train loss
box 0.16111931204795837
obj 0.2353966385126114
noobj 1.100995421409607
class 0.9086500406265259

train loss
box 0.15015526115894318
obj 0.15270721912384033
noobj 1.3191837072372437
class 0.8829957246780396

train loss
box 0.2900596261024475
obj 0.254782497882843
noobj 0.46108365058898926
class 0.9890755414962769

train loss
box 0.13355812430381775
obj 0.2576735019683838
noobj 1.1202611923217773
class 0.7967932224273682

train loss
box 0.12183976173400879
obj 0.1523982137441635
noobj 1.3314790725708008
class 0.6814407110214233

train loss
box 0.3

100%|██████████| 12/12 [00:22<00:00,  1.84s/it]

Class accuracy is: 89.057121%
No obj accuracy is: 98.625473%
Obj accuracy is: 59.233768%


In [ ]:
save_checkpoint(model, optimizer, filename="highobj.pth.tar")

=> Saving checkpoint
